# Base class

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import tqdm
from datetime import datetime

class simulation():
  def __init__(self,params):
    self.box_length = params['box_length']
    self.num_particles = params['num_particles']
    self.num_steps = params['num_steps']
    self.len_time_step = params['len_time_step']
    self.disk_radius = params['disk_radius']
    self.f_active = params['f_active']
    self.diffusion = params['diffusion']
    self.rot_diffusion = params['rot_diffusion']
    self.frame_length_multiplier = params['frame_length_multiplier']
    self.scanning_length = 0

    self.B = np.sqrt(2 * self.diffusion)
    self.C = np.sqrt(2 * self.rot_diffusion * self.len_time_step)

    self.positions_init = np.random.uniform(0,self.box_length,(self.num_particles,2))
    self.angles_init = np.random.uniform(0,2 * np.pi,(self.num_particles,1))

    self.particle_velocities = []

  def step(self,positions,angles):
    xi = np.random.normal(0,1,(self.num_particles,2))
    xi_r = np.random.normal(0,1,(self.num_particles,1))

    drive = self.f_active * np.hstack((np.cos(angles),np.sin(angles)))
    repulsion = self.calc_repulsion(positions)

    dr = ((repulsion + drive) + self.B * xi )* self.len_time_step
    self.velocity = dr
    d_angles = self.C * xi_r

    new_positions = (positions + dr) % self.box_length
    new_angles = (angles + d_angles)

    return new_positions, new_angles

  def simulate(self): # done
    record = []
    positions = self.positions_init
    angles = self.angles_init

    loading_bar = tqdm.tqdm(total=self.num_steps, desc='simulation progress', position=0)

    for i in range(self.num_steps):
      record.append(positions)
      [positions,angles] = self.step(positions,angles)
      loading_bar.update(1)
    return record

  def animate(self):
    self.run()

    fig, ax = plt.subplots(figsize = (self.box_length, self.box_length))
    ax.set(xlim=[0, self.box_length], ylim=[0, self.box_length])
    ax.set_aspect('equal')
    self.fig_size = fig.get_size_inches() * fig.dpi

    self.scat = ax.scatter(self.record[0][:,0],self.record[0][:,1],s=self.rectify_radius(self.disk_radius, self.box_length), color = "lightgreen", edgecolor = "black")

    ani = animation.FuncAnimation(fig=fig,func=self.update_frame,frames = self.num_steps,interval=self.len_time_step*1000 * self.frame_length_multiplier)

    now = datetime.now()
    timestr = now.strftime("%Y-%m-%d %H:%M:%S")
    ani.save(f'{type(self).__name__} {timestr}.mp4')

  def rectify_radius(self,particleradius, boxlength):
      scaledradius = particleradius * (int(self.fig_size[0]) / boxlength)
      s_value = 0.4 * np.pi * scaledradius**2
      return s_value

  def update_frame(self,frame):
      data = self.record[frame]
      data = np.stack([data[:,0],data[:,1]]).T
      self.scat.set_offsets(data)
      return self.scat

  def calc_repulsion(self, positions):
    LM = positions[positions[:, 0] > (self.box_length - self.scanning_length)] - np.array([self.box_length, 0])
    RM = positions[positions[:, 0] < self.scanning_length] + np.array([self.box_length, 0])
    MD = positions[positions[:, 1] > (self.box_length - self.scanning_length)] - np.array([0, self.box_length])
    MU = positions[positions[:, 1] < 2 * self.scanning_length] + np.array([0, self.box_length])

    mirror_positions = np.vstack((positions, MD, LM, RM, MU))
    diff = positions[:, None, :] - mirror_positions[None, :, :]
    distances = np.linalg.norm(diff, axis=2)

    #Stops 0 error
    distances = np.maximum(distances, 1e-6)
    within_range = distances < self.scanning_length

    #What optimizes
    rep_magnitudes = np.zeros_like(distances)
    rep_magnitudes[within_range] = self.repulsion_magnitude(distances[within_range])
    normalized_diff = diff / distances[..., None]
    repulsion = np.sum(rep_magnitudes[..., None] * normalized_diff, axis=1)
    return repulsion

  def repulsion_magnitude(self,r):
    return 0

  def run(self):
    self.record = self.simulate()

  def graph_bond_order(self):
    record = self.record
    bond_record = np.zeros(self.num_steps)
    bond_record_individual = []
    n_crystallized = np.zeros(self.num_steps)
    crystal_threshold = 0.8

    loading_bar1 = tqdm.tqdm(total=self.num_steps, desc='graph progress', position=0)
    for i in range(self.num_steps):
      psi6,q6 = self.calc_bond_order(record[i])
      bond_record[i] = psi6
      bond_record_individual.append(q6)
      n_crystallized[i] = np.count_nonzero(q6 > crystal_threshold)
      loading_bar1.update(1)

    fig,ax = plt.subplots()
    ax.plot(range(self.num_steps),bond_record)
    ax.set_title('global bond orientation order')
    ax.set_ylabel('$\psi_6$')
    ax.set_xlabel('time step')

    fig1,ax1 = plt.subplots()
    ax1.hist(bond_record_individual[-1],bins=30)
    ax1.set_title(f'crystallization at t = {self.num_steps}')
    ax1.set_xlabel('bond orientation order')
    ax1.set_ylabel('frequency')

    fig2,ax2 = plt.subplots()
    ax2.plot(range(self.num_steps),n_crystallized)
    ax2.set_title(f'particles with bond order > {crystal_threshold}')
    ax2.set_xlabel('time step')
    ax2.set_ylabel('number of particles')

  def calc_bond_order(self,positions):
    q6 = np.zeros(self.num_particles,dtype=np.complex128)

    for i in range(self.num_particles):
        diff = positions - positions[i]
        distances = np.linalg.norm(diff, axis=1)
        neighbors_idx = np.argsort(distances)[1:7]
        bond_angles = np.arctan2(diff[neighbors_idx, 1], diff[neighbors_idx, 0])
        q6[i] = np.mean(np.exp(1j * 6 * bond_angles))
        psi6 = np.abs(np.mean(q6))**2
    return psi6, np.abs(q6)

  def make_velocity_plots(self):
    square_speeds = [np.sum(self.particle_velocities[i] **2,axis=1) for i in range(10,self.num_steps)]
    avg_sq_speed = np.sum(square_speeds,axis=1)/self.num_particles

    fig,ax = plt.subplots()
    ax.plot(range(10,self.num_steps),avg_sq_speed)
    ax.set(ylim=[0, 2*np.mean(avg_sq_speed)]) # do labeling things
    ax.set_title('Mean squared velocity')
    ax.set_xlabel('time step')

    fig1,ax1 = plt.subplots()
    ax1.hist(square_speeds[-1]**0.5,bins=30)
    ax1.set_title(f'speed distribution at t = {self.num_steps}')
    ax1.set_xlabel('speed')
    ax1.set_ylabel('frequency')

###

class contact_simulation(simulation):
  def __init__(self,params):
    super().__init__(params)
    self.scanning_length = 2 * self.disk_radius
    self.k = params['k']

  def repulsion_magnitude(self,r):
      return self.k * (2*self.disk_radius - r)

class yukawa_simulation(simulation):
  def __init__(self,params):
    super().__init__(params)
    self.Gamma = params['Gamma']
    self.Lambda = params['Lambda']
    self.scanning_length = 7 / self.Lambda
  def repulsion_magnitude(self, r):
      return (self.Gamma * np.exp(-self.Lambda * r) / r**2 ) + (self.Gamma * self.Lambda * np.exp(-self.Lambda * r)/r)

# just animation

In [ ]:
params = {'box_length':20,'num_particles':200,'num_steps':500,'len_time_step':1e-3,'disk_radius':0.5,'k':100,'f_active':7,'diffusion':3,'rot_diffusion':3,'frame_length_multiplier':3}

sim = contact_simulation(params)
sim.animate()

In [ ]:
params = {'box_length':20,'num_particles':400,'num_steps':300,'len_time_step':1e-4,'disk_radius':0.35,'f_active':15,'diffusion':3.5,'rot_diffusion':3.5,'Gamma':500,'Lambda':3.5,'frame_length_multiplier':100}

ysim = yukawa_simulation(params)
ysim.animate()

In [ ]:
params = {'box_length':20,'num_particles':400,'num_steps':300,'len_time_step':1e-4,'disk_radius':0.35,'f_active':15,'diffusion':3.5,'rot_diffusion':3.5,'Gamma':500,'Lambda':3.5,'frame_length_multiplier':100}

class yukawa_simulation(simulation):
  def __init__(self,params):
    super().__init__(params)
    self.Gamma = params['Gamma']
    self.Lambda = params['Lambda']
    self.scanning_length = 7 / self.Lambda
  def repulsion_magnitude(self, r):
      return (self.Gamma * np.exp(-self.Lambda * r) / r**2 ) + (self.Gamma * self.Lambda * np.exp(-self.Lambda * r)/r)

# entropic force

In [ ]:
class entropic_force_simulation(simulation):
  def __init__(self,params):
    super().__init__(params)
    self.positions_init = np.full((self.num_particles,2),self.box_length / 2) # all start in the middle
    self.scanning_length = 0

  def avg_radius(self):
    avg_radius = np.zeros(self.num_steps)
    for i in range(self.num_steps):
      avg_radius[i] = np.mean(np.sqrt(np.sum((self.record[i]-self.positions_init)**2,axis=1)))

    fig,ax = plt.subplots()
    ax.plot(range(self.num_steps),avg_radius)
    ax.set_title('average particle radius from center')

params = {'box_length':20,'num_particles':100,'num_steps':1000,'len_time_step':1e-3,'disk_radius':0.35,'f_active':0,'diffusion':10000,'rot_diffusion':3,'frame_length_multiplier':50}

esim = entropic_force_simulation(params)
esim.animate()
esim.avg_radius()

# bond orientation order

In [ ]:
params = {'box_length':20,'num_particles':400,'num_steps':2000,'len_time_step':1e-4,'disk_radius':0.35,'f_active':7,'diffusion':3.5,'rot_diffusion':3.5,'Gamma':500,'Lambda':3.5,'frame_length_multiplier':100}

ygsim = yukawa_simulation(params)
ygsim.run()
ygsim.graph_bond_order()

In [ ]:
params = {'box_length':30,'num_particles':800,'num_steps':2000,'len_time_step':5e-4,'disk_radius':0.5,'k':600,'f_active':20,'diffusion':3,'rot_diffusion':3,'frame_length_multiplier':50}

sim = contact_simulation(params)
sim.animate()
sim.graph_bond_order()

In [ ]:
# control

params = {'box_length':15,'num_particles':200,'num_steps':500,'len_time_step':1e-3,'disk_radius':0.5,'k':100,'f_active':7,'diffusion':3,'rot_diffusion':3,'frame_length_multiplier':3}

sim = simulation(params)
sim.run()
sim.graph_bond_order()

# velocity

In [ ]:
class contact_velocity_graphing(contact_simulation):
  def __init__(self,params):
    super().__init__(params)
    self.particle_velocities = [] # M x (n x 2)

  def step(self,positions,angles):
    new_positions,new_angles = super().step(positions,angles)
    self.particle_velocities.append(self.velocity)
    return new_positions,new_angles

params = {'box_length':40,'num_particles':1000,'num_steps':500,'len_time_step':1e-3,'disk_radius':0.5,'k':100,'f_active':7,'diffusion':3,'rot_diffusion':3,'frame_length_multiplier':3}

sim = contact_velocity_graphing(params)
sim.run()
sim.make_velocity_plots()

In [ ]:
class yukawa_velocity_graphing(yukawa_simulation):
  def __init__(self,params):
    super().__init__(params)
    self.particle_velocities = []

  def step(self,positions,angles):
    new_positions,new_angles = super().step(positions,angles)
    self.particle_velocities.append(self.velocity)
    return new_positions,new_angles

params = {'box_length':40,'num_particles':1000,'num_steps':1000,'len_time_step':1e-4,'disk_radius':0.35,'f_active':15,'diffusion':3.5,'rot_diffusion':3.5,'Gamma':500,'Lambda':3.5,'frame_length_multiplier':100}

yvgsim = yukawa_velocity_graphing(params)
yvgsim.run()
yvgsim.make_velocity_plots()